In [1]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [2]:
italy_gdf = gpd.read_file(
    "data/ondata_confini_amministrativi_api_v2_it_20250101_unita-territoriali-sovracomunali.geo.json"
)

In [3]:
italy_gdf = italy_gdf[["den_uts", "geometry"]]
italy_gdf = italy_gdf.rename(columns={"den_uts": "Province"})

In [4]:
to_replace = {
    "Aosta": "Valle d'Aosta/Vallée d'Aoste",
    "Bolzano": "Bolzano/Bozen",
    "Forli'-Cesena": "Forlì-Cesena",
    "Massa Carrara": "Massa-Carrara",
}
italy_gdf = italy_gdf.replace(to_replace)

## Get Open Population data

In [5]:
population_2025_df = pd.read_csv(
    "data/Resident population.csv",
)

In [6]:
population_2025_df = population_2025_df[["Province", "Total"]]
population_2025_df["Total"] = population_2025_df["Total"].astype(int)

In [7]:
population_2019_df = pd.read_csv(
    "data/Resident population_2019.csv",
)

In [8]:
population_2019_df = population_2019_df[["Province", "Total"]]
population_2019_df["Total"] = population_2019_df["Total"].astype(int)

In [9]:
population_df = population_2019_df.merge(
    population_2025_df, on="Province", how="inner", suffixes=("_2019", "_2025")
)

In [10]:
population_df["DIFF"] = population_df["Total_2025"] - population_df["Total_2019"]
population_df["PERCENT"] = (
    (population_df["DIFF"] / population_df["Total_2019"]) * 100
).round(decimals=1)

## Merge data and export

In [16]:
italy_pop_df = italy_gdf.merge(population_df, on="Province", how="inner")

In [17]:
italy_pop_df.to_file("data/italy_pop.gpkg")

In [19]:
italy_pop_df["Total_2025"].sum()

np.int64(58934177)

In [20]:
italy_pop_df["Total_2019"].sum()

np.int64(59816673)